In [1]:
## trying to obtain go directly from the foudner population to evaluate extintction risk 
import pandas as pd

/tmp/ipykernel_2011127/121802926.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
## last gen 
from scipy.stats import spearmanr, pearsonr, kendalltau

import pandas as pd
import pickle
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ecotypes_names = pd.read_csv('../key_files/founder_ecotype_names.csv')['0'].to_list()

In [4]:
clim_sites_during_exp = pd.read_csv('../key_files/bioclimvars_sites_era5_year_2018.csv')


In [5]:
first_gen = pd.read_csv('../key_files/generation_1_sample_names.txt',header=None)[0]
samples = first_gen.to_list()

In [6]:
unique_sites = first_gen.str.split('_').str[0].unique()

In [7]:
#sort_bio1 = clim_sites_during_exp
clim_sites_during_exp_s = clim_sites_during_exp[clim_sites_during_exp['site'].isin(unique_sites)]

sort_bio1 = clim_sites_during_exp_s[['site', 'bio1']].sort_values('bio1')['site'].astype(str).values

In [8]:
sort_bio1_int = clim_sites_during_exp_s[['site', 'bio1']].sort_values('bio1')['site'].values

In [9]:
delta_ef=pd.read_csv('../key_files/delta_ecotype_freq.txt', sep = '\t', usecols = samples)#[['log_p1_p0','ecotype', 'site','plot']]

ef=pd.read_csv('../key_files/merged_ecotype_frequency.txt', sep = '\t', usecols = samples)#[['log_p1_p0','ecotype', 'site','plot']]
delta_ef.index = ecotypes_names
ef.index = ecotypes_names

In [10]:
file_path = '../jacknife_first_gen/splits_samples_first_gen.pkl'
# splits_samples_last_gen_no33.pkl
# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples = pickle.load(file)

In [12]:
genotype_counts = pd.read_csv('../leave_1_out/genotype_counts_fullgenome231ecotypes.csv').drop('Unnamed: 0', axis=1)

In [13]:
snp_dict = pd.read_csv('../key_files/var_pos_grenenet.csv')

In [14]:
pd.read_csv('../key_files/dict_snps.csv')

,id,pos,chrom,maf05filter,total_alleles05filter
0,1_291,291,1,NaN,NaN
1,1_346,346,1,1_346,1_346
2,1_349,349,1,NaN,NaN
3,1_352,352,1,NaN,NaN
4,1_353,353,1,1_353,NaN
...,...,...,...,...,...
3235475,5_26975208,26975208,5,NaN,NaN
3235476,5_26975215,26975215,5,NaN,NaN
3235477,5_26975267,26975267,5,NaN,NaN
3235478,5_26975272,26975272,5,5_26975272,5_26975272


In [30]:
allele_sums = genotype_counts.sum(axis=1)

# Calculate the total number of alleles per SNP (2 * number of individuals)
num_genotypes = genotype_counts.shape[1]
total_alleles = 2 * num_genotypes

# Calculate allele frequencies
allele_frequencies_founders = allele_sums / total_alleles

In [31]:
allele_frequencies_founders

0          0.043290
1          0.272727
2          0.008658
3          0.021645
4          0.069264
             ...   
3235475    0.012987
3235476    0.008658
3235477    0.004329
3235478    0.385281
3235479    0.216450
Length: 3235480, dtype: float64

In [17]:
split = 2

In [39]:
br_results = pd.read_csv(f'../binomial_regression/binomial_reg_results_bio1_prediction_allsites.csv')

In [40]:
##filter for significance 
#th = 0.05 / len(br_results['pvalue'])
#br_results = br_results[br_results['pvalue'] < th]

In [46]:
sites_avail = br_results.columns[4:-1]

In [51]:
# go_per_site all sites used for trianing 
go_per_site = {}
for i in sites_avail:
    site = i.split('_')[-1]
    mask_full_genotype = snp_dict['id'].isin(br_results['snp_id'])
    allele_frequencies_founders = allele_frequencies_founders.loc[mask_full_genotype]
    
    #allele_frequencies_founders = allele_frequencies_founders.reset_index(drop=True)

    site_predict_proba = br_results[i]
    
    ## for each site, subrstract the actual value by the probability fo getting a success 
    allele_frequencies_founders_sub = allele_frequencies_founders.sub(site_predict_proba, axis=0)
    allele_frequencies_founders_sub = allele_frequencies_founders_sub**2
    ## ge tthe absolute value 
    
    ## sum over ecotypes 
    go = (allele_frequencies_founders_sub.sum(axis=0)) / len(allele_frequencies_founders_sub)
    go_per_site[site] = go

In [52]:
go_per_site_df = pd.DataFrame.from_dict(go_per_site, orient='index', columns=['value'])

go_per_site_df = go_per_site_df.reset_index()

go_per_site_df.columns = ['site', 'go']

In [53]:
go_per_site_df.to_csv('go_from_founder_pop_all_sites_intraining.csv',index=None)

In [32]:
# go_per_site leave 1 our 

go_per_site = {}
for split in range(len(samples)):
    print(split)
    br_results = pd.read_csv(f'results/split_{split}/binomial_reg_results_last_gen.csv')
    
    br_results = br_results.reset_index(drop=True)

    ##filter for significance 
    th = 0.05 / len(br_results['pvalue'])
    br_results = br_results[br_results['pvalue'] < th]

    
    mask_full_genotype = snp_dict['id'].isin(br_results['snp_id'])
    allele_frequencies_founders = allele_frequencies_founders.loc[mask_full_genotype]
    
    #allele_frequencies_founders = allele_frequencies_founders.reset_index(drop=True)
    
    site_predicted = br_results.columns[0]
    print(site_predicted)
    site_predict_proba = br_results[site_predicted]
    
    ## for each site, subrstract the actual value by the probability fo getting a success 
    allele_frequencies_founders_sub = allele_frequencies_founders.sub(site_predict_proba, axis=0)
    allele_frequencies_founders_sub = allele_frequencies_founders_sub**2
    ## ge tthe absolute value 
    
    ## sum over ecotypes 
    go = (allele_frequencies_founders_sub.sum(axis=0)) / len(allele_frequencies_founders_sub)
    go_per_site[site_predicted] = go

0
1
1
2
2
4
3
5
4
6
5
9
6
10
7
11
8
12
9
13
10
23
11
24
12
25
13
26
14
27
15
28
16
32
17
33
18
37
19
42
20
43
21
45
22
46
23
48
24
49
25
52
26
53
27
54
28
55
29
57
30
60


In [32]:
go_per_site

{'1': 0.02109333355414395,
 '2': 0.021248224178853185,
 '4': 0.021281258744652132,
 '5': 0.021212978617400093,
 '6': 0.02121395369772065,
 '9': 0.02115840616613045,
 '10': 0.021522961856719758,
 '11': 0.021199854902575956,
 '12': 0.02116360839527905,
 '13': 0.021476620614141666,
 '23': 0.020966450919219056,
 '24': 0.02095129557041857,
 '25': 0.021096562058456277,
 '26': 0.021643325719795763,
 '27': 0.020967175804635342,
 '28': 0.021301733513674655,
 '32': 0.021243961649562344,
 '33': 0.021110212725121445,
 '37': 0.020990137916659025,
 '42': 0.021053046350214696,
 '43': 0.021353577200608074,
 '45': 0.02126489366047693,
 '46': 0.021105683859776292,
 '48': 0.02102031123233362,
 '49': 0.021101300989315806,
 '52': 0.021119563731284974,
 '53': 0.021130630637124902,
 '54': 0.021121238350940718,
 '55': 0.021091441776055223,
 '57': 0.021139448203186275,
 '60': 0.021407395755464898}

In [ ]:
go_per_site_df = pd.DataFrame.from_dict(go_per_site, orient='index', columns=['value'])

go_per_site_df = go_per_site_df.reset_index()

go_per_site_df.columns = ['site', 'go']

go_per_site_df.to_csv('go_sign_snps_from_founder_pop_l1o.csv',index=None)

In [ ]:
## for the extinct sites i wi;; use the prob of al lteh other sites 

In [38]:
go_per_site_df

,site,go
0,1,0.010975
1,2,0.011065
2,4,0.010138
3,5,0.010236
4,6,0.010300
5,9,0.010235
6,10,0.010850
7,11,0.010319
8,12,0.010282
9,13,0.010595


In [17]:
file_path = '../jacknife_first_gen/fake_splits_samples_for_unseen_sites.pkl'
# Open and load the .pkl file
with open(file_path, 'rb') as file:
    samples_fake = pickle.load(file)

In [18]:
go_per_site_unseen = {}
for split in range(len(samples_fake)):
    print(split)
    br_results = pd.read_csv(f'results_sites_unseen/split_{split}/binomial_reg_results_first_gen_sites_unseen.csv')
    
    br_results = br_results.reset_index(drop=True)
    
    mask_full_genotype = snp_dict['id'].isin(br_results['snp_id'])
    
    allele_frequencies_founders = allele_frequencies_founders.loc[mask_full_genotype]
    
    #allele_frequencies_founders = allele_frequencies_founders.reset_index(drop=True)
    
    site_predicted = br_results.columns[0]
    print(site_predicted)
    site_predict_proba = br_results[site_predicted]
    
    ## for each site, subrstract the actual value by the probability fo getting a success 
    allele_frequencies_founders_sub = allele_frequencies_founders.sub(site_predict_proba, axis=0)
    allele_frequencies_founders_sub = allele_frequencies_founders_sub**2
    ## ge tthe absolute value 
    
    ## sum over ecotypes 
    go = (allele_frequencies_founders_sub.sum(axis=0)) / len(allele_frequencies_founders_sub)
    go_per_site_unseen[site_predicted] = go

0
7
1
50
2
56
3
21
4
41
5
47


In [19]:
go_per_site_unseen

{'7': 0.021082786062428523,
 '50': 0.021029196511582617,
 '56': 0.020965680099733087,
 '21': 0.021072923058000098,
 '41': 0.02108183000416506,
 '47': 0.02112429904009548}

In [30]:
go_per_site_unseen_df = pd.DataFrame.from_dict(go_per_site_unseen, orient='index', columns=['value'])

In [31]:
go_per_site_unseen_df = go_per_site_unseen_df.reset_index()

In [33]:
go_per_site = pd.concat([go_per_site_unseen_df, go_per_site_df])

In [35]:
go_per_site.columns = ['site', 'go']

In [37]:
go_per_site.to_csv('go_w_exctintsites.csv',index=None)

In [52]:
go_per_site_df.to_csv('genomic_offset_per_site_firt_gen_prob_squared_l1out_all_proba.csv',index=None)